## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
import sys
sys.path.append('C:\\Users\\David\\repos\\data_repos\\World_Weather_Analysis')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Oruro,BO,-17.9833,-67.1500,57.63,14,0,6.64,clear sky
1,1,Khatanga,RU,71.9667,102.5000,48.04,87,3,14.05,clear sky
2,2,Yellowknife,CA,62.4560,-114.3525,70.32,43,75,20.71,broken clouds
3,3,Hilo,US,19.7297,-155.0900,84.27,95,100,8.05,moderate rain
4,4,Anadyr,RU,64.7500,177.4833,46.90,75,20,6.71,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
filtered_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Hilo,US,19.7297,-155.0900,84.27,95,100,8.05,moderate rain
5,5,Baijiantan,CN,45.6333,85.1833,81.34,28,20,11.52,few clouds
6,6,Jamestown,US,42.0970,-79.2353,80.51,65,75,18.41,broken clouds
7,7,Mapiripan,CO,2.8892,-72.1256,79.90,86,72,6.96,broken clouds
10,10,Flagstaff,US,35.1981,-111.6513,88.56,32,20,8.05,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.count()

City_ID                180
City                   180
Country                180
Lat                    180
Lng                    180
Max Temp               180
Humidity               180
Cloudiness             180
Wind Speed             180
Current Description    180
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_cities_df = filtered_cities_df.dropna()
filtered_cities_df.count() # No rows were dropped since no values were empty.

City_ID                180
City                   180
Country                180
Lat                    180
Lng                    180
Max Temp               180
Humidity               180
Cloudiness             180
Wind Speed             180
Current Description    180
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hilo,US,84.27,moderate rain,19.7297,-155.0900,
5,Baijiantan,CN,81.34,few clouds,45.6333,85.1833,
6,Jamestown,US,80.51,broken clouds,42.0970,-79.2353,
7,Mapiripan,CO,79.90,broken clouds,2.8892,-72.1256,
10,Flagstaff,US,88.56,few clouds,35.1981,-111.6513,
13,Kapaa,US,82.72,few clouds,22.0752,-159.3190,
19,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,
22,Piacabucu,BR,75.56,broken clouds,-10.4056,-36.4344,
23,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,
27,Atar,MR,83.35,moderate rain,20.5169,-13.0499,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... Skipping.")


Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("", np.nan, inplace = True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hilo,US,84.27,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Jamestown,US,80.51,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
7,Mapiripan,CO,79.90,broken clouds,2.8892,-72.1256,Monserrate
10,Flagstaff,US,88.56,few clouds,35.1981,-111.6513,Drury Inn & Suites Flagstaff
13,Kapaa,US,82.72,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
19,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn
22,Piacabucu,BR,75.56,broken clouds,-10.4056,-36.4344,Piaçabuçu News
23,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
27,Atar,MR,83.35,moderate rain,20.5169,-13.0499,Oumou Elghoura
29,Torbay,CA,81.88,broken clouds,47.6666,-52.7314,Gallows Cove


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))